## Задание:

Проанализировать датафрейм статистики NBA. (файл nba.csv) Вывести:

1. Проранжировать бюджет команд на зарплату от самого высокого до самого низкого.
2. 10 наиболее высокооплачиваемых игроков в лиге.
3. 3 наиболее высооплачиваемых игроков в каждой команде.
4. Предложить первые пятерки каждой команды на основе их зарплаты (учитывайте, что в первой пятерке на каждой позиции один игрок (см. рисунок)).
5. Самая молодая и самая возростная команда в лиге.
6. Наиболее часто используемый номер игрока в лиге.
7. Имя и рост самого высокого и самого низкого игрока в лиге в метрической системе.

P.S. Возможные пропуски в зарплатной ведомости заполнить медианным значением зарплаты команды игрока с пропуском.

### Подготовка данных:

Загружаем данные, попутно конвертируя номер игрока в строку, возраст в целое число и рост в метрическую систему.

In [1]:
import pandas as pd

def float2str(n):
    return str(int(float(n) if n > '' else 0))

def float2int(n):
    return int(float(n) if n > '' else 0)

def imp2metric(n):
    if n > '':
        arr = n.split('-')
        if len(arr) == 2:
            return int(round(int(arr[0]) * 30.48 + int(arr[1]) * 2.54))
        else:
            return 0
    else:
        return 0

whole_data = pd.read_csv('nba.csv', 
                         delimiter = ',', 
                         converters = { 
                             'Number': float2str,  
                             'Age': float2int,
                             'Height': imp2metric
                         })


Строки без имени игрока не несут никакой информации. Убираем:

In [2]:
data = whole_data[whole_data['Name'] > '']

Посмотрим, где пусто:

In [3]:
print(data.isnull().sum())

Name         0
Team         0
Number       0
Position     0
Age          0
Height       0
Weight       0
College     84
Salary      11
dtype: int64


Колледж нас не интересует, а зарплату заполним медианным значением, взятым по команде.

In [4]:
grouped = data.groupby('Team')
res = []
for team in data.Team.unique():
    by_team = grouped.get_group(team)
    median = by_team['Salary'].median()    
    res.append(by_team.fillna({ 'Salary': median }))
data = pd.concat(res)    


### Задание 1:

Бюджеты команд в порядке убывания:

In [5]:
grouped = data.groupby('Team')
res = []
for team in data.Team.unique():
    by_team = grouped.get_group(team)
    res.append({ 'Team': team, 'Salary': by_team['Salary'].sum() })   
res.sort(key = lambda n: n['Salary'], reverse = True)    
print(*res, sep='\n')

{'Team': 'Cleveland Cavaliers', 'Salary': 111963689.0}
{'Team': 'Los Angeles Clippers', 'Salary': 94854640.0}
{'Team': 'Memphis Grizzlies', 'Salary': 94726918.0}
{'Team': 'Oklahoma City Thunder', 'Salary': 93765298.0}
{'Team': 'Golden State Warriors', 'Salary': 88868997.0}
{'Team': 'Miami Heat', 'Salary': 87479113.0}
{'Team': 'Chicago Bulls', 'Salary': 86783378.0}
{'Team': 'San Antonio Spurs', 'Salary': 84442733.0}
{'Team': 'New Orleans Pelicans', 'Salary': 82750774.0}
{'Team': 'Charlotte Hornets', 'Salary': 78340920.0}
{'Team': 'Washington Wizards', 'Salary': 76328636.0}
{'Team': 'Houston Rockets', 'Salary': 75283021.0}
{'Team': 'New York Knicks', 'Salary': 73303898.0}
{'Team': 'Atlanta Hawks', 'Salary': 72902950.0}
{'Team': 'Los Angeles Lakers', 'Salary': 71770431.0}
{'Team': 'Sacramento Kings', 'Salary': 71683666.0}
{'Team': 'Dallas Mavericks', 'Salary': 71198732.0}
{'Team': 'Toronto Raptors', 'Salary': 71117611.0}
{'Team': 'Milwaukee Bucks', 'Salary': 69603517.0}
{'Team': 'Detroit 

### Задание 2:

10 наиболее высокооплачиваемых игроков в лиге.

In [6]:
data.sort_values('Salary', inplace = True, ascending = False)
print(data.head(10)[['Name', 'Salary']])

                Name      Salary
109      Kobe Bryant  25000000.0
169     LeBron James  22970500.0
33   Carmelo Anthony  22875000.0
251    Dwight Howard  22359364.0
339       Chris Bosh  22192730.0
100       Chris Paul  21468695.0
414     Kevin Durant  20158622.0
164     Derrick Rose  20093064.0
349      Dwyane Wade  20000000.0
174       Kevin Love  19689000.0


### Задание 3:
3 наиболее высооплачиваемых игрока в каждой команде.

In [7]:
grouped = data.groupby('Team')
for team in data.Team.unique():
    by_team = grouped.get_group(team).sort_values('Salary', ascending = False)
    print(team)
    print(by_team.head(3)[['Name', 'Salary']])
    print('\n')


Los Angeles Lakers
               Name      Salary
109     Kobe Bryant  25000000.0
111     Roy Hibbert  15592217.0
118  Louis Williams   7000000.0


Cleveland Cavaliers
             Name      Salary
169  LeBron James  22970500.0
174    Kevin Love  19689000.0
168  Kyrie Irving  16407501.0


New York Knicks
               Name      Salary
33  Carmelo Anthony  22875000.0
38      Robin Lopez  12650000.0
30    Arron Afflalo   8000000.0


Houston Rockets
              Name      Salary
251  Dwight Howard  22359364.0
249   James Harden  15756438.0
245   Corey Brewer   8229375.0


Miami Heat
             Name      Salary
339    Chris Bosh  22192730.0
349   Dwyane Wade  20000000.0
341  Goran Dragic  14783000.0


Los Angeles Clippers
               Name      Salary
100      Chris Paul  21468695.0
98   DeAndre Jordan  19689000.0
96    Blake Griffin  18907726.0


Oklahoma City Thunder
                  Name      Salary
414       Kevin Durant  20158622.0
426  Russell Westbrook  16744218.0
418       

### Задание 4:
Предложить первые пятерки каждой команды на основе их зарплаты (учитывайте, что в первой пятерке на каждой позиции один игрок (см. рисунок)).

In [13]:
positions = data.Position.unique()
grouped = data.groupby('Team')
for team in data.Team.unique():
    by_team = grouped.get_group(team).sort_values('Salary', ascending = False)
    print(team)
    print('=========================')
    for position in positions:
      print('{0} -- {1}'.format(position, by_team[by_team.Position == position].iloc[0]['Name']))        
    print('\n') 
    
    


Boston Celtics
PG -- Avery Bradley
SG -- Evan Turner
SF -- Jae Crowder
PF -- Amir Johnson
C -- Tyler Zeller


Phoenix Suns
PG -- Eric Bledsoe
SG -- Devin Booker
SF -- P.J. Tucker
PF -- Mirza Teletovic
C -- Tyson Chandler


Indiana Pacers
PG -- George Hill
SG -- Monta Ellis
SF -- Paul George
PF -- Lavoy Allen
C -- Jordan Hill


Brooklyn Nets
PG -- Jarrett Jack
SG -- Bojan Bogdanovic


IndexError: single positional indexer is out-of-bounds

### Задание 5:
Самая молодая и самая возростная команда в лиге.

In [9]:
grouped = data.groupby('Team')
res = []
for team in data.Team.unique():
    by_team = grouped.get_group(team)
    res.append([team, round(by_team['Age'].mean(), 1)])
res.sort(key = lambda n: n[1])    
print('Самая молодая: {0}, средний возраст: {1}'.format(res[0][0], res[0][1]))
print('Самая возрастная: {0}, средний возраст: {1}'.format(res[len(res) - 1][0], res[len(res) - 1][1]))


Самая молодая: Utah Jazz, средний возраст: 24.5
Самая возрастная: San Antonio Spurs, средний возраст: 31.6


### Задание 6:
Наиболее часто используемый номер игрока в лиге.

In [10]:
frequent_number = data['Number'].mode().iloc[0]
print(frequent_number)

5


### Задание 7:
Имя и рост самого высокого и самого низкого игрока в лиге в метрической системе.

In [11]:
data.sort_values(by = 'Height', inplace = True)  
print('Самый высокий: {0}, рост: {1} см'.format(data.tail(1).iloc[0]['Name'], data.tail(1).iloc[0]['Height']))
print('Самый низкий: {0}, рост: {1} см'.format(data.head(1).iloc[0]['Name'], data.head(1).iloc[0]['Height']))


Самый высокий: Walter Tavares, рост: 221 см
Самый низкий: Isaiah Thomas, рост: 175 см
